# First approach

In [ ]:
# Preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedStratifiedKFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler, FunctionTransformer, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import VotingClassifier, StackingClassifier

# Target encoding/decoding
from sklearn.base import BaseEstimator, TransformerMixin

# Metrics
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, auc, roc_curve, log_loss

# Models
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier, plot_importance
from catboost import CatBoostClassifier

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Math and DataFrame
import pandas as pd
import numpy as np

# Warnings ignore
import warnings
warnings.filterwarnings("ignore")

In [ ]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')
original = pd.read_csv('/content/steel_plates_faults_original_dataset.csv')
train.head()

In [ ]:
train.shape

In [ ]:
train = pd.concat([train, original], axis=0).drop_duplicates()
train.head()

In [ ]:
train.shape

In [ ]:
train = train.drop('id', axis=1)
test = test.drop('id', axis=1)
train.head()

In [ ]:
target_variables = ['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps',  'Other_Faults']

In [ ]:
# If the sum of the axis is greater than 1 because in the same row more than 1 columns value is 1, we can sure that is not a multi class problem, it is multi label problem.
no_faults = train[train[target_variables].sum(axis=1) == 0][target_variables]
print(f'\nNumber of No Faults: {no_faults.shape[0]}\n')
no_faults

In [ ]:
no_faults.shape[0] / train.shape[0] * 100

In [ ]:
more_than_one_faults = train[train[target_variables].sum(axis=1) > 1][target_variables]
print(f'\nNumber of More than one faults: {more_than_one_faults.shape[0]}\n')
more_than_one_faults

In [ ]:
train = train[train[target_variables].sum(axis=1) <= 1]
train

In [ ]:
# Create a numpy array from our given targets.
targets_arr = train[target_variables].values.copy()
targets_arr

In [ ]:
no_faults_arr = 1 - targets_arr.sum(axis=1)[:, np.newaxis]
print(f'\nNumber of the No Faults: {no_faults_arr.sum()}\n')
no_faults_arr

In [ ]:
targets_concatenated = np.concatenate([targets_arr, no_faults_arr], axis=1)
targets_concatenated

In [ ]:
X = train.drop(target_variables, axis=1)
target = train[target_variables]
X.head()

In [ ]:
scaler = StandardScaler()
X_sc = scaler.fit_transform(X)
test_sc = scaler.transform(test)
X_sc

### One-hot encoded to label-encoded

In [ ]:
label_encoded_targets = np.argmax(targets_concatenated, axis=-1)
label_encoded_targets

In [ ]:
unique_classes = np.unique(label_encoded_targets)
unique_classes

In [ ]:
class_weights = compute_class_weight(class_weight="balanced", classes=unique_classes, y=label_encoded_targets)
class_weights

In [ ]:
class_weights_param = {key: value for key, value in zip(unique_classes, class_weights)}
class_weights_param

### Optuna tuned parameters

In [ ]:
TUNE = False

In [ ]:
import optuna

# Define the objective function for Optuna optimization
def objective(trial, X_train, y_train, X_test, y_test):
    # Define parameters to be optimized for the LGBMClassifier
    param = {
        "class_weight": class_weights_param,
        "objective": "multiclass",
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "random_state": 42,
        "num_class": 7,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.03),
        "n_estimators": trial.suggest_int("n_estimators", 400, 600),
        "lambda_l1": trial.suggest_float("lambda_l1", 0.005, 0.025),
        "lambda_l2": trial.suggest_float("lambda_l2", 0.02, 0.06),
        "max_depth": trial.suggest_int("max_depth", 6, 14),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.3, 0.9),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 50),
    }

    # Create an instance of LGBMClassifier with the suggested parameters
    lgbm_classifier = LGBMClassifier(**param)

    # Fit the classifier on the training data
    lgbm_classifier.fit(X_train, y_train)

    # Evaluate the classifier on the test data
    score = lgbm_classifier.score(X_test, y_test)

    print(f'SCORE: {score}')

    return score

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_sc, label_encoded_targets, test_size=0.2, random_state=42)  # Adjust the test_size as needed

# Set up the sampler for Optuna optimization
sampler = optuna.samplers.TPESampler(seed=42)  # Using Tree-structured Parzen Estimator sampler for optimization

# Create a study object for Optuna optimization
study = optuna.create_study(direction="maximize", sampler=sampler)

# If TUNE
if TUNE:
    # Run the optimization process
    study.optimize(lambda trial: objective(trial, X_train, y_train, X_test, y_test), n_trials=150)

    # Get the best parameters after optimization
    best_params = study.best_params

In [ ]:
# Define the objective function for Optuna optimization
def objective(trial, X_train, y_train, X_test, y_test):
    # Define parameters to be optimized for the XGBClassifier
    param = {
        "objective": 'multi:softmax',
        "booster": 'gbtree',
        "random_state": 42,
        "num_class": 7,
        'n_estimators': trial.suggest_int('n_estimators', 400, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
        'gamma' : trial.suggest_float('gamma', 1e-9, 1.0),
        'subsample': trial.suggest_float('subsample', 0.25, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.25, 1.0),
        'max_depth': trial.suggest_int('max_depth', 0, 24),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 30),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-9, 10.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-9, 10.0, log=True),
    }

    # Create an instance of LGBMClassifier with the suggested parameters
    xgb_classifier = XGBClassifier(**param)

    # Fit the classifier on the training data
    xgb_classifier.fit(X_train, y_train)

    # Evaluate the classifier on the test data
    score = xgb_classifier.score(X_test, y_test)
    y_pred_prob = xgb_classifier.predict_proba(X_test)
    lgbm_log_loss = log_loss(y_test, y_pred_prob)
    print(f'SCORE: {score}')
    print(f'Log Loss: {lgbm_log_loss}')

    return score

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_sc, label_encoded_targets, test_size=0.2, random_state=42)  # Adjust the test_size as needed

# Set up the sampler for Optuna optimization
sampler = optuna.samplers.TPESampler(seed=42)  # Using Tree-structured Parzen Estimator sampler for optimization

# Create a study object for Optuna optimization
study = optuna.create_study(direction="maximize", sampler=sampler)

if TUNE:

    # Run the optimization process
    study.optimize(lambda trial: objective(trial, X_train, y_train, X_test, y_test), n_trials=150)

    # Get the best parameters after optimization
    best_params = study.best_params


In [ ]:
lgbm_params: dict = {
    "class_weight": class_weights_param, # Balanced class weight
    "objective": "multiclass",          # Objective function for the model
    "metric": "multi_logloss",          # Evaluation metric
    "verbosity": -1,                    # Verbosity level (-1 for silent)
    "boosting_type": "gbdt",            # Gradient boosting type
    "random_state": 42,       # Random state for reproducibility
    "num_class": 7,                     # Number of classes in the dataset
    'n_estimators': 752,
    'learning_rate': 0.005613916463106189,
    'max_depth': 6,
    'num_leaves': 252,
    'subsample': 0.6045538705062335,
    'colsample_bytree': 0.866501494211133,
    'colsample_bynode': 0.5443098861233086,
    'reg_alpha': 0.0024787490924597882,
    'reg_lambda': 3.5334079815178954,
    'min_split_gain': 0.05539710161546875,
}

In [ ]:
xgb_params: dict = {
    'class_weight': class_weights_param,
    'objective':'multi:softmax',
    'n_estimators': 829,
    'learning_rate': 0.010260565670497695,
    'gamma': 0.16282691057583543,
    'reg_alpha': 0.010492176264956674,
    'reg_lambda': 0.437536781187624,
    'max_depth': 5,
    'min_child_weight': 2,
    'subsample': 0.6971737476610285,
    'colsample_bytree': 0.5115061295805807,
    'random_state': 345,
}

In [ ]:
cat_params: dict = {
    'class_weights': class_weights_param,
    'learning_rate': 0.13762007048684638,
    'depth': 5,
    'l2_leaf_reg': 5.285199432056192,
    'bagging_temperature': 0.6029582154263095,
    'random_seed': 42,
    'verbose': False,
    'iterations':1000,
}

In [ ]:
estimators = [
    ('XGB', XGBClassifier(**xgb_params)),
    ('LGBM', LGBMClassifier(**lgbm_params)),
    ('CAT', CatBoostClassifier(**cat_params))
]

In [ ]:
WEIGHT_TUNE = False

In [ ]:
# Define the objective function for Optuna optimization
def objective(trial, X_train, y_train, X_test, y_test):
    # Define parameters to be optimized for the weighted ensemble

    obj_estimators = [
        ('XGB', XGBClassifier(**xgb_params)),
        ('LGBM', LGBMClassifier(**lgbm_params)),
        ('CAT', CatBoostClassifier(**cat_params))
    ]

    voting_classifier = VotingClassifier(
        estimators=obj_estimators,
        voting='soft',
        weights=[
            trial.suggest_float('XGB_Weight', 1.00, 9.00),
            trial.suggest_float('LGBM_Weight', 0.25, 5.00),
            trial.suggest_float('CAT_Weight', 0.25, 2.00),
        ],
    )

    voting_classifier.fit(X_train, y_train)

    predict_probs = voting_classifier.predict_proba(X_test)

    auc_score = roc_auc_score(y_test, predict_probs, multi_class='ovr')

    return auc_score

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_sc, label_encoded_targets, test_size=0.2, random_state=42)  # Adjust the test_size as needed

# Set up the sampler for Optuna optimization
weight_sampler = optuna.samplers.TPESampler(seed=42)  # Using Tree-structured Parzen Estimator sampler for optimization

# Create a study object for Optuna optimization
weight_study = optuna.create_study(direction="maximize", sampler=weight_sampler)

if WEIGHT_TUNE:

    # Run the optimization process
    weight_study.optimize(lambda trial: objective(trial, X_train, y_train, X_test, y_test), n_trials=300)

    # Get the best parameters after optimization
    weight_best_params = weight_study.best_params

In [ ]:
voting_estimators = [
    ('XGB', XGBClassifier(**xgb_params)),
    ('LGBM', LGBMClassifier(**lgbm_params)),
    ('CAT', CatBoostClassifier(**cat_params))
]

In [ ]:
voting_classifier = VotingClassifier(
    estimators=voting_estimators,
    voting='soft',
    weights=[
            7.994761950304625,
            0.46883567511191715,
            0.4412142916220983
        ],
    )

voting_classifier.fit(X_sc, label_encoded_targets)

predicted_probs = voting_classifier.predict_proba(test_sc)[:, :-1] # Filtering out 8th's target (No Faults)

In [ ]:
predicted_probs.shape

In [ ]:
submission_df = pd.read_csv('/kaggle/input/playground-series-s4e3/sample_submission.csv')
submission_df

submission_df[target_variables] = predicted_probs
submission_df

In [ ]:
submission_df.to_csv("submission.csv", index=False)

# Second approach

In [ ]:
import sklearn
import numpy as np
import os
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno
from prettytable import PrettyTable
%matplotlib inline
import seaborn as sns
sns.set(style='darkgrid', font_scale=1.4)
from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm_notebook
tqdm_notebook.get_lock().locks = []
# !pip install sweetviz
# import sweetviz as sv
import concurrent.futures
from copy import deepcopy
from functools import partial
from itertools import combinations
import random
from random import randint, uniform
import gc
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler,PowerTransformer, FunctionTransformer
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from itertools import combinations
from sklearn.impute import SimpleImputer
import xgboost as xg
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import mean_squared_error,mean_squared_log_error, roc_auc_score, accuracy_score, f1_score, precision_recall_curve, log_loss
from sklearn.cluster import KMeans
!pip install yellowbrick
from yellowbrick.cluster import KElbowVisualizer
!pip install gap-stat
from gap_statistic.optimalK import OptimalK
from scipy import stats
import statsmodels.api as sm
from scipy.stats import ttest_ind
from scipy.stats import boxcox
import math
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.base import BaseEstimator, TransformerMixin
!pip install optuna
import optuna
!pip install cmaes
import cmaes
import xgboost as xgb
!pip install catboost
!pip install lightgbm --install-option=--gpu --install-option="--boost-root=C:/local/boost_1_69_0" --install-option="--boost-librarydir=C:/local/boost_1_69_0/lib64-msvc-14.1"
import lightgbm as lgb
!pip install category_encoders
from category_encoders import OneHotEncoder, OrdinalEncoder, CountEncoder, CatBoostEncoder
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier,ExtraTreesClassifier, AdaBoostClassifier
!pip install -U imbalanced-learn
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoost, CatBoostRegressor, CatBoostClassifier
from sklearn.svm import NuSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import KNNImputer
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from catboost import Pool
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")
pd.pandas.set_option('display.max_columns',None)

In [ ]:
train=pd.read_csv('/kaggle/input/playground-series-s4e3/train.csv')
test=pd.read_csv('/kaggle/input/playground-series-s4e3/test.csv')
original=pd.read_csv("/kaggle/input/steel-plates-faults/SteelPlatesFaults.csv")

train.drop(columns=["id"],inplace=True)
test.drop(columns=["id"],inplace=True)

train_copy=train.copy()
test_copy=test.copy()
original_copy=original.copy()

print(original.shape)

device='cpu'

train=pd.concat([train,original],axis=0)
train.reset_index(inplace=True,drop=True)

target=['Pastry','Z_Scratch','K_Scatch','Stains','Dirtiness','Bumps','Other_Faults']

original.head()

### Exploratory analysis

In [ ]:
cont_cols = test.columns
colors = ['blue', 'orange', 'green']

num_plots = len(cont_cols)
num_cols = 3
num_rows = -(-num_plots // num_cols)
fig, axes = plt.subplots(num_rows, num_cols, figsize=(21, 5 * num_rows))  # Adjust the figure size as needed

for i, feature in enumerate(cont_cols):
    row = i // num_cols
    col = i % num_cols

    ax = axes[row, col] if num_rows > 1 else axes[col]

    sns.histplot(train_copy[feature], kde=True, color=colors[0], label='Train', alpha=0.5, bins=30, ax=ax)
    sns.histplot(test_copy[feature], kde=True, color=colors[1], label='Test', alpha=0.5, bins=30, ax=ax)
    sns.histplot(original[feature], kde=True, color=colors[2], label='Original', alpha=0.5, bins=30, ax=ax)

    ax.set_title(f'Distribution of {feature}')
    ax.set_xlabel(feature)
    ax.set_ylabel('Frequency')
    ax.legend()

if num_plots % num_cols != 0:
    for j in range(num_plots % num_cols, num_cols):
        axes[-1, j].axis('off')

plt.tight_layout()
plt.show()

### Feature engineering

In [ ]:
def OHE(train_df,test_df,cols,target):
    '''
    Function for one hot encoding, it first combined the data so that no category is missed and
    the category with least frequency can be dropped because of redunancy
    '''
    combined = pd.concat([train_df, test_df], axis=0)
    for col in cols:
        one_hot = pd.get_dummies(combined[col]).astype(int)
        counts = combined[col].value_counts()
        min_count_category = counts.idxmin()
        one_hot = one_hot.drop(min_count_category, axis=1)
        one_hot.columns=[str(f)+col for f in one_hot.columns]
        combined = pd.concat([combined, one_hot], axis="columns")
        combined = combined.loc[:, ~combined.columns.duplicated()]

    # split back to train and test dataframes
    train_ohe = combined[:len(train_df)]
    test_ohe = combined[len(train_df):]
    test_ohe.reset_index(inplace=True,drop=True)
    test_ohe.drop(columns=[target],inplace=True)
    return train_ohe, test_ohe